In [267]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import math

In [204]:
tickers = ['VTI', 'AGG', 'DBC', '^VIX']
data = yf.download(tickers, start="2011-01-01", end="2020-04-30", interval="1d")['Adj Close']
data_na = data.dropna(axis = 0)
data_na.dropna(axis=0, inplace=True)
data_na.reset_index(inplace=True)
data_na['Date'] = data_na['Date'].dt.date


[*********************100%***********************]  4 of 4 completed


In [205]:
data_na['Date'] = pd.to_datetime(data_na['Date'])
initial_allocation = [0.25, 0.25, 0.25, 0.25]

In [206]:
# Allocation initiale
initial_allocation = [0.25, 0.25, 0.25, 0.25]  # Allocations initiales pour VTI, AGG, DBC, VIX
portfolio_value = 1000


In [207]:
# Créer un DataFrame pour les allocations initiales
allocation_df = pd.DataFrame(index=data_na.index, columns=[f'{col}_alloc' for col in data_na.columns[1:]])

# Initialiser les allocations du portefeuille pour chaque actif
initial_prices = data_na.iloc[0, 1:]  # Prix initiaux des actifs
allocation_df.iloc[0] = [portfolio_value * alloc / price for alloc, price in zip(initial_allocation, initial_prices)]


In [208]:
def rebalance_portfolio(row, prev_alloc, prices, initial_allocation):
    total_value = (prev_alloc * prices).sum()  # Calcul de la valeur totale du portefeuille
    new_alloc = [(total_value * alloc) / price for alloc, price in zip(initial_allocation, prices)]
    return pd.Series(new_alloc)


In [210]:
# DataFrame pour stocker les valeurs du portefeuille
portfolio_df = pd.DataFrame(index=data_na.index, columns=['Date', 'portfolio_value'])
portfolio_df['Date'] = data_na['Date']

In [211]:
# Parcourir chaque année et réallouer le portefeuille au 1er janvier de chaque année
for i in range(1, len(data_na)):
    current_year = pd.to_datetime(data_na.loc[i, 'Date']).year
    previous_year = pd.to_datetime(data_na.loc[i-1, 'Date']).year
    
    if current_year != previous_year:  # Réallocation si on change d'année
        previous_alloc = allocation_df.iloc[i-1].values.astype(float)
        current_prices = data_na.iloc[i, 1:].values.astype(float)
        
        # Réallocation basée sur les prix actuels et les allocations initiales
        allocation_df.iloc[i] = rebalance_portfolio(data_na.iloc[i],previous_alloc, current_prices, initial_allocation)
    else:
        allocation_df.iloc[i] = allocation_df.iloc[i-1]  # Conserver la même allocation
    
    # Utilisation de loc pour éviter les avertissements
    portfolio_df.loc[i, 'portfolio_value'] = (allocation_df.iloc[i] * data_na.iloc[i, 1:].values).sum()


In [189]:
# Calculer les poids du portefeuille (allocation en pourcentage du portefeuille total)
weights_df = pd.DataFrame(index=allocation_df.index, columns=[f'{col}_weight' for col in allocation_df.columns[0:]])

for i in range(len(data_na)):
    total_value = portfolio_df['portfolio_value'].iloc[i]
    # Convertir allocation en tableau numpy pour le calcul des poids
    current_alloc = allocation_df.iloc[i].values.astype(float)
    weights_df.iloc[i] = (current_alloc * data_na.iloc[i, 1:].values) / total_value

In [219]:
# Initialisation du DataFrame des poids
weights_df = pd.DataFrame(index=data_na.index, columns=data_na.columns[1:])

# Poids fixes pour 2011
weights_2011 = [0.25] * len(data_na.columns[1:])  # 0.25 pour chaque actif

# Assignation des poids pour 2011
for i in range(len(data_na)):
    current_year = pd.to_datetime(data_na.loc[i, 'Date']).year
    
    if current_year == 2011:
        weights_df.loc[i] = weights_2011  # Assigner 0.25 pour 2011
    elif current_year > 2011:
        # Si c'est la première occurrence de l'année
        if i == 0 or pd.to_datetime(data_na.loc[i - 1, 'Date']).year != current_year:
            # Récupérer les prix de la première ligne de l'année
            first_row_prices = data_na.iloc[i, 1:].values.astype(float)
            # Calculer les poids basés sur la première ligne de l'année
            weights_df.loc[i] = fixed_weights = first_row_prices / first_row_prices.sum()
        else:
            # Pour les autres jours de la même année, conserver les mêmes poids
            weights_df.loc[i] = weights_df.loc[i - 1]



In [223]:
# Supprimer la dernière ligne du DataFrame des poids
weights_df = weights_df.drop(weights_df.index[-1]).reset_index(drop=True)


In [195]:
# Calculer les rendements quotidiens
returns_df = data_na.iloc[:, 1:].pct_change().fillna(0)
returns_df['Date'] = data_na['Date']
returns_df = returns_df[['Date'] + list(returns_df.columns[:-1])]



In [ ]:
returns_df = returns_df.drop(index=returns_df.index[0]).reset_index(drop=True)

In [236]:
returns_df = returns_df.drop(columns=['Date'])


In [238]:
returns_df

Ticker,AGG,DBC,VTI,^VIX
0,0.000285,-0.013372,-0.001982,-0.013061
1,-0.004827,0.010623,0.005499,-0.020713
2,0.000190,-0.013411,-0.002278,0.022327
3,0.003233,-0.003674,-0.002284,-0.014943
4,0.002654,0.011799,0.000763,0.023337
...,...,...,...,...
2340,0.002390,-0.002760,0.000143,-0.014292
2341,-0.000511,-0.012915,0.014905,-0.131706
2342,-0.004601,-0.018692,0.017722,-0.073476
2343,0.003938,0.000952,-0.003122,0.008411


In [255]:
weights_df.iloc[252]

Ticker
AGG     0.441186
DBC     0.141729
VTI     0.289137
^VIX    0.127948
Name: 252, dtype: object

In [258]:
returns_df

Ticker,AGG,DBC,VTI,^VIX
0,0.000285,-0.013372,-0.001982,-0.013061
1,-0.004827,0.010623,0.005499,-0.020713
2,0.000190,-0.013411,-0.002278,0.022327
3,0.003233,-0.003674,-0.002284,-0.014943
4,0.002654,0.011799,0.000763,0.023337
...,...,...,...,...
2340,0.002390,-0.002760,0.000143,-0.014292
2341,-0.000511,-0.012915,0.014905,-0.131706
2342,-0.004601,-0.018692,0.017722,-0.073476
2343,0.003938,0.000952,-0.003122,0.008411


In [260]:
weights_df 


Ticker,AGG,DBC,VTI,^VIX
0,0.25,0.25,0.25,0.25
1,0.25,0.25,0.25,0.25
2,0.25,0.25,0.25,0.25
3,0.25,0.25,0.25,0.25
4,0.25,0.25,0.25,0.25
...,...,...,...,...
2340,0.354704,0.053944,0.546893,0.044458
2341,0.354704,0.053944,0.546893,0.044458
2342,0.354704,0.053944,0.546893,0.044458
2343,0.354704,0.053944,0.546893,0.044458


In [244]:
# Calculer le produit terme à terme
weighted_returns = returns_df.values * weights_df.values

# Somme sur les lignes pour obtenir le rendement du portefeuille
portfolio_returns = weighted_returns.sum(axis=1)

In [257]:
portfolio_returns

array([-0.007032314008862395, -0.0023548570675245573,
       0.0017068397352348608, ..., 0.003785344700315989,
       0.00011441350426329898, 0.01360897261630044], dtype=object)

In [268]:
# Calcul de l'espérance des rendements (moyenne)
expected_return = portfolio_returns.mean()
expected_return = (expected_return+1)**252-1

# Calcul de la standard déviation
std_dev = portfolio_returns.std()
std_dev = std_dev*math.sqrt(252)
risk_free_rate = 0.0

# Calcul du Sharpe ratio
sharpe_ratio = (expected_return - risk_free_rate) / std_dev

# Affichage des résultats
print(f"Espérance des rendements : {expected_return:.4f}")
print(f"Standard déviation : {std_dev:.4f}")
print(f"Sharpe ratio : {sharpe_ratio:.4f}")


Espérance des rendements : 0.1390
Standard déviation : 0.1238
Sharpe ratio : 1.1229


In [263]:
C = 0.01  

# Initialisation du vecteur des coûts de transaction
transaction_costs = np.zeros(len(weights_df))

# Calcul des coûts de transaction
for t in range(2, len(weights_df)):
    # Calcul des différences de poids
    diff_weights = weights_df.iloc[t - 1].values - weights_df.iloc[t - 2].values
    
    # Somme des valeurs absolues des différences
    sum_abs_diff = np.sum(np.abs(diff_weights))
    
    # Coût de transaction pour le temps t
    transaction_costs[t] = C * sum_abs_diff

transaction_costs

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# Convertir le vecteur en DataFrame ou Series si nécessaire
transaction_costs_df = pd.Series(transaction_costs, index=data_na['Date'].dt.date)

# Affichage des coûts de transaction
print("Vecteur des coûts de transaction :")
print(transaction_costs_df.head(10))  # Affiche les 10 premières valeurs
